In [4]:
import pandas as pd
import pandas_datareader as dr
import numpy as np 
import yfinance as yf

snp500url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
data_tab = pd.read_html(snp500url)

tickers = data_tab[0][1:]['Symbol'].tolist()

print("Total number of tickers", len(tickers))

raw = yf.download(tickers, start="2022-01-01", end="2025-01-01", auto_adjust=True)

Total number of tickers 502


[*********************100%***********************]  502 of 502 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2022-01-01 -> 2025-01-01)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [ ]:
dataset = pd.DataFrame(raw)
missing_frac = dataset.isnull().mean().sort_values(ascending=False)
drop_list = sorted(list(missing_frac[missing_frac > 0.2].index))
dataset.drop(columns=drop_list, axis = 1, inplace=True)
dataset.fillna(method='bfill', axis='index', inplace=True)
print('Null values:', dataset.isnull().values.any())
dataset.head()

In [ ]:
import quantstats as qs

def data_vertical(data):
    dataset_vertical = data.stack(level=1).reset_index()
    dataset_vertical.rename(columns={'level_1': 'Ticker'}, inplace=True)
    dataset_vertical['Date'] = pd.to_datetime(dataset_vertical['Date'])  # Ensure 'Date' column exists
    dataset_vertical.set_index('Date', inplace=True)
    return dataset_vertical
    


def feature_engineering(data):
    #avg returns
    returns = data["Close"].pct_change().mean()*252
    returns = pd.DataFrame(returns)
    final_dataframe = returns.reset_index()
    final_dataframe = final_dataframe.rename(columns={final_dataframe.columns[1]:"Yavg_return"})
    #volatility
    final_dataframe["Yavg_volatility"] = np.array(data["Close"].pct_change().std()*np.sqrt(252))

    dataset_vertical = data_vertical(data)
    dataset_vertical["daily_span"] = dataset_vertical["High"]- dataset_vertical["Low"]
    #daily_span
    final_dataframe["Davg_span"] = np.array(dataset_vertical.groupby("Ticker")["daily_span"].mean())
    #traded volume
    final_dataframe["Davg_volume"] = np.array(dataset_vertical.groupby("Ticker")["Volume"].mean())
    #skewness
    final_dataframe["D_eSkewness"] = np.array(dataset_vertical.groupby("Ticker")["Close"].apply(qs.stats.skew))

    #VaR
    final_dataframe["D_eVaR"] = np.array(dataset_vertical.groupby("Ticker")["Close"].apply(qs.stats.value_at_risk))
    #CVaR
    final_dataframe["D_eCVaR"] = np.array(dataset_vertical.groupby("Ticker")["Close"].apply(qs.stats.expected_shortfall))
    #Curtosis
    final_dataframe["D_eCurtosis"] = np.array(dataset_vertical.groupby("Ticker")["Close"].apply(qs.stats.kurtosis))

    return final_dataframe
fdata = feature_engineering(dataset).head()

C:\Users\m.narese\AppData\Local\Temp\ipykernel_19464\1583726913.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  dataset_vertical = data.stack(level=1).reset_index()


In [100]:
from analysis.data.utils_analysis import create_full_dataset
ESG = pd.read_csv(r"C:\Users\m.narese\Desktop\THESIS\REPO\portfolio_optimization\analysis\datasets\1\ESG_data.csv")
stock_data = create_full_dataset(fdata, ESG)
stock_data.drop(columns=["logo", "name", "weburl", "exchange", "last_processing_date", "cik"])

,Ticker,Yavg_return,Yavg_volatility,Davg_span,Davg_volume,eSkewness,eVaR,eCVaR,eCurtosis,currency,...,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,total_grade,total_level
0,A,-0.000397,0.296259,3.088187,1.753392e+06,0.153981,-0.030678,-0.039708,2.402057,USD,...,B,Medium,B,Medium,573,231,233,1037,BBB,High
1,AAPL,0.148965,0.270935,3.567855,6.808211e+07,0.213657,-0.027464,-0.037961,2.522160,USD,...,B,Medium,B,Medium,355,281,255,891,BB,Medium
2,ABBV,0.153699,0.219958,2.795256,5.845151e+06,-1.449195,-0.022167,-0.036429,12.029841,USD,...,BB,Medium,BB,Medium,505,317,300,1122,BBB,High
3,ABNB,0.018549,0.467875,4.557586,5.709665e+06,-0.243738,-0.048374,-0.069937,2.304313,USD,...,A,High,BBB,High,505,570,400,1475,A,High
4,ABT,-0.026648,0.217839,1.901359,5.428663e+06,0.145626,-0.022662,-0.030448,2.618757,USD,...,BB,Medium,BB,Medium,515,305,305,1125,BBB,High
